In [23]:
urls = ['https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx',
        'https://www.koreabaseball.com/Record/Player/HitterBasic/Basic2.aspx',
        'https://www.koreabaseball.com/Record/Player/HitterBasic/Detail1.aspx']
driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx')
# driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/Basic2.aspx')
# driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/Detail1.aspx')

In [26]:
def base_click(max_attempts, driver):
    time.sleep(1)
    for attempt in range(max_attempts):
        try:
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.tab-depth3 > ul > li:nth-child(1) > a'))).click()
            break
        except StaleElementReferenceException as e:
            if attempt == max_attempts - 1:
                raise
            time.sleep(.5)
        except ElementClickInterceptedException as e:
            print('기본 기록 클릭할 수 없음')
            if attempt == max_attempts - 1:
                raise
            time.sleep(1)

def next_click(max_attempts, driver):
    time.sleep(1)
    for attempt in range(max_attempts):
        try:
            # wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.row > div.more_record > a.next'))).click()
            next_record = driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.row > div.more_record > a.next')
            driver.execute_script('arguments[0].click()', next_record)
            break
        except StaleElementReferenceException as e:
            if attempt == max_attempts - 1:
                raise
            time.sleep(.5)
        except ElementClickInterceptedException as e:
            print('다음 기록 클릭할 수 없음')
            if attempt == max_attempts - 1:
                raise
            time.sleep(1)
    
def detail_click(max_attempts, driver):
    time.sleep(1)
    for attempt in range(max_attempts):
        try:
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.row > div.tab-depth3 > ul > li:nth-child(2) > a'))).click()
            break
        except StaleElementReferenceException as e:
            if attempt == max_attempts - 1:
                raise
            time.sleep(.5)
        except ElementClickInterceptedException as e:
            print('세부 기록 클릭할 수 없음')
            if attempt == max_attempts - 1:
                raise
            time.sleep(1)

In [27]:
def page_click(driver):
    # 페이지 개수 확인
    page_count = len(driver.find_elements(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.record_result > div > a'))
    # 페이지 크롤링
    df = create_table(driver)

    if page_count > 1:
        time.sleep(.2)
        # 다음 페이지 넘어가기
        driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ucPager_btnNo2').click()
        time.sleep(.2)
        # 페이지 크롤
        df2 = create_table(driver)
        # 1, 2페이지 데이터 합치기
        df = pd.concat([df, df2])
        time.sleep(.2)
        # 1 페이지로 돌아가기
        driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ucPager_btnNo1').click()
        
    return df

In [ ]:
def page_click(driver):
    page_count = len(driver.find_elements(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.record_result > div > a'))
    
    df1 = create_table(driver)
    time.sleep(.2)
    driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.row > div.more_record > a.next').click()
    df2 = create_table(driver)
    time.sleep(.2)
    driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.row > div.tab-depth3 > ul > li:nth-child(2) > a').click()
    df3 = create_table(driver)
    
    page_1_df = pd.concat([df1, df2, df3], axis=1)
    
    if page_count > 1:
        driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ucPager_btnNo2').click()
        df1 = create_table(driver)
        time.sleep(.2)
        driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.row > div.more_record > a.next').click()
        time.sleep(.2)
        driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ucPager_btnNo2').click()
        df2 = create_table(driver)
        time.sleep(.2)
        driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.row > div.tab-depth3 > ul > li:nth-child(2) > a').click()
        time.sleep(.2)
        driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ucPager_btnNo2').click()
        df3 = create_table(driver)
        
        page_2_df = pd.concat([df1, df2, df3], axis=1)
        
        return pd.concat([page_1_df, page_2_df])
    else:
        return page_1_df
        

In [28]:
# 페이지 로드를 기다림
def wait_for_page_to_load(driver):
    wait = WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")

In [29]:
record_dfs = []
seasons = [str(i) for i in range(2002, 2025)]

for url in urls:
    driver.get(url)
    dfs = []
    for season in seasons:    
        # 시즌 콤보 박스 선택
        season_combo = driver.find_element(By.CSS_SELECTOR, '#cphContents_cDwdphContents_cphContents_ddlSeason_ddlSeason')
        # 시즌 콤보 박스 선택
        season_combo = Select(season_combo)
        # 값 선택
        season_combo.select_by_value(season)
        teams = team_list(driver)
        for team in teams:
            time.sleep(.5)
            combobox = driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlTeam_ddlTeam')
            team_combo = Select(combobox)
            team_combo.select_by_visible_text(team)
            time.sleep(.5)
            df = page_click(driver)
            df['year'] = season 
            dfs.append(df)
        time.sleep(10)
        print(season,'년 완료!')
    record_dfs.append(dfs)
    
# 인덱스 안맞음 방지
df1 = pd.concat(record_dfs[0]).reset_index()
df2 = pd.concat(record_dfs[1]).reset_index()
df3 = pd.concat(record_dfs[2]).reset_index()

df = pd.concat([df1, df2, df3], axis=1)
# 필요없는 데이터 제거
df.drop(['BB/K볼넷/삼진', 'GO땅볼', 'ISOP순수장타율', 'GO/AO땅볼/뜬공', 'HR홈런', 'OBP출루율', 'BB볼넷'], axis=1, inplace=True)
df.loc[:, ~df.columns.duplicated()].to_csv('2002_2024.csv', encoding='cp949')

/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tab

2002 년 완료!


/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tab

2003 년 완료!


/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tab

...

2022 년 완료!


/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tab

2023 년 완료!


/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table), flavor='html5lib')[0]
/tmp/ipykernel_312/1424746438.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tab

2024 년 완료!


In [64]:
df1 = pd.concat(record_dfs[0]).reset_index()
df2 = pd.concat(record_dfs[1]).reset_index()
df3 = pd.concat(record_dfs[2]).reset_index()

/tmp/ipykernel_312/4027143814.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df2 = pd.concat(record_dfs[1]).reset_index()
/tmp/ipykernel_312/4027143814.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df3 = pd.concat(record_dfs[2]).reset_index()


In [83]:
df = pd.concat([df1, df2, df3], axis=1)
df.drop(['BB/K볼넷/삼진', 'GO땅볼', 'ISOP순수장타율', 'GO/AO땅볼/뜬공', 'HR홈런', 'OBP출루율', 'BB볼넷'], axis=1, inplace=True)
df.loc[:, ~df.columns.duplicated()].to_csv('2002_2024.csv', encoding='cp949')

In [1]:
!pip freeze > requirements.txt